In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter, MarkdownHeaderTextSplitter, MarkdownTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever 
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
import time
import re
from konlpy.tag import Okt
from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
from PyPDF2 import PdfReader
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask

def get_embedding():
    
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text

def format_docs(docs):
    context = ""
    
    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header{header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context


In [3]:
def process_pdf(pdf_path): 
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
        ("######", "Header 6")
    ]

def chunk_documents(docs):
    
    
    # 마크다운 스플리터 설정
    markdown_splitter = RecursiveCharacterTextSplitter(
        separators = [
                # First, try to split along Markdown headings (starting with level 2)
                "\n#{1,6} ",
                # Note the alternative syntax for headings (below) is not handled here
                # Heading level 2
                # ---------------
                # End of code block
                "```\n",
                # Horizontal lines
                "\n\\*\\*\\*+\n",
                "\n---+\n",
                "\n___+\n",
                # Note that this splitter doesn't handle horizontal lines defined
                # by *three or more* of ***, ---, or ___, but this is not handled
                "\n\n",
                "\n",
                " ",
                "",
            ],
        chunk_size=800,
        chunk_overlap=80,
    )

    splits = text_splitter.split_documents(md_chunks)

    for i in splits:
        metadata = {'Source_path': pdf_path}
        i.metadata = {**i.metadata, **metadata}
    return splits


def make_db(df):
    documents = []
    
    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(process_pdf(pdf_file))
        
    print(f"Total number of documents: {len(documents)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    return faiss

In [4]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [6]:
def format_docs(docs):
    context = ""
    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header {header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context

### Model

In [7]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    
    # 일부 중요한 레이어는 FP16으로 유지
    for name, module in model.named_modules():
        if "attention" in name or "ffn" in name:  # 중요한 레이어 식별 (예: attention 및 ffn)
            module.to(torch.float16)  # 이 부분은 16비트로 유지
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
# ghost-x/ghost-8b-beta-1608
# OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.02s/it]


### 점수

In [8]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

### RUN

In [9]:

def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def equal_path(contexts, source_path):
    adjusted_docs = []
    for doc in contexts:
        if doc.metadata['Source_path'] == source_path:
            adjusted_docs.append(doc)
    return adjusted_docs

def rerun(question,context,answer,llm,num_repeat):
    full_template = "<|begin_of_text|>"
    full_template += """<|start_header_id|>system<|end_header_id|>당신은 이전 답변을 검증하는 챗봇입니다. 질문과 문맥, 이전 답변을 참고해서 지시사항을 따르세요. 지시사항을 따를 때 서론 없이 출력하세요.<|eot_id|>"""
    full_template += f"""<|start_header_id|>user<|end_header_id|>Question: {question} \n\nContexts: {context} \n\nPrevious Answer: {answer} \n\n"""
    full_template += """{input}<|eot_id|>"""
    full_template += """<|start_header_id|>assistant<|end_header_id|>"""
    
    prompt = PromptTemplate(template=full_template)
    chain = (
    {
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    return chain.invoke("핵심 단어들을 바탕으로, 한 문장으로 요약하세요. 만약 한 문장이라면 그대로 출력하세요.")
    
def run(faiss,dataset,llm,k=4,verbose=False):
    results = []
    source_path = dataset.iloc[0]['Source_path']
    docs = faiss.similarity_search(
    query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
    filter={"Source_path": source_path},
    k = 99,
    fetch_k = 20000
    )
    buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
    faiss_retriever = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k,'fetch_k': 50})
    bm_retriever = OktBM25Retriever.from_documents(docs)
    bm_retriever.k = k
    ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever,bm_retriever], weight=[0.4, 0.3, 0.3])
    
    for i, row in (dataset.iterrows()):
        if source_path != row['Source_path']:   
            source_path = row['Source_path']
            docs = faiss.similarity_searchfaiss.similarity_search(
                query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
                filter={"Source_path": source_path},
                k = 99,
                fetch_k = 20000
            )
            buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
            faiss_retriever = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k,'fetch_k': 50})
            bm_retriever = OktBM25Retriever.from_documents(docs)
            bm_retriever.k = k
            ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever,bm_retriever], weight=[0.4, 0.3, 0.3])
            
        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
당신은 유용한 금융 정보 QnA 챗봇입니다.
질문을 천천히 이해 해보세요.
객관적이고 공식적인 문체를 사용하세요.
답변 시 반드시 문맥 정보를 활용해야합니다. 
서론 없이 핵심 내용을 한 문장으로 작성해주세요. 
<|eot_id|>
"""
        question = row['Question']          
        # full_template += """ """
        contexts = ensemble_retriever.invoke(normalize_string(question))
        # contexts = equal_path(contexts,row['Source_path'])
        contexts = format_docs(contexts)
        full_template += """<|start_header_id|>user<|end_header_id|>Question: {input}\n\n"""
        full_template += f"""Contexts: {contexts}<|eot_id|>"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["input"])
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        answer = extract_answer(answer)
        lines = answer.split('\n')
        if  len(lines) > 1:
            previous = answer
            try:
                before = calculate_f1_score(row['Answer'],answer)[2]
            except:
                before = None
            answer = rerun(question=question,
                           context=contexts,
                           answer=answer,
                           llm=llm,
                           num_repeat=1)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            try:
                print(calculate_f1_score(row['Answer'],answer)[2],end=" | ")
            except:
                pass
            print("Answer: ", results[-1]['Answer'])
            try:
                print("Before: ",before," | ",previous)  
                
                previous = None
                before = None
            except:
                pass
            
            try:
                print("REAL Answer: ",row['Answer'])
            except:
                pass
            
            print()

    return results

### 케이폴드

In [10]:
from save_module import save

faiss= make_db(test_df)

results = run(faiss, test_df, llm, verbose=True)
save(results)

100%|██████████| 14/14 [02:29<00:00, 10.70s/it]


Total number of documents: 1071


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:653: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


1/124
Question:  2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요? | Context Number | 4946
0.9166666666666665 | Answer:  2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 특별회계 81.7조원으로 구성됩니다.
REAL Answer:  2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.

4/124
Question:  2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요? | Context Number | 2713
0.9494949494949494 | Answer:  2024년 총수입은 612.2조원입니다. 예산수입은 395.5조원이며, 기금수입은 216.7조원입니다.
REAL Answer:  2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216.7조원입니다.

6/124
Question:  2024년의 기금수입은 어떻게 구성되어 있나요? | Context Number | 2827
0.7839999999999999 | Answer:  2024년의 기금수입은 총 216.7조원으로, 사회보장기여금 92.3조원, 융자원금회수 33.8조원, 기타 90.6조원으로 구성되어 있습니다.
REAL Answer:  2024년도 기금수입은 사회보장성기금 92.3조원, 경상이전수입 39.6조원, 기타 84.7조원으로 구성되어 있습니다.

11/124
Question:  2024년 총지출 기준 예산의 일반회계와 특별회계의 비중이 각각 얼마인가? | Context Number | 5423
0.775 | Answer:  2024년 총지출 기준 예산의 일반회계와 특별회계의 비중은 각각 87.3%와 9.9%입니다.
REAL Answer:  2024년 총지출 중 일반회계와 특별회계의 비중은 각각 54.3%와 12.4%이다.

19/124
Question:  2020년 결산 기준, 연구개발비 중 개발연구 규모는 몇 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


...

299/124
Question:  생계급여와 관련된 법령상 근거는 무엇인가? | Context Number | 4298
0.8974358974358975 | Answer:  생계급여와 관련된 법령상 근거는 「국민기초생활보장법」 제7조 ~ 제10조입니다.
Before:  None  |  None
REAL Answer:  생계급여와 관련된 법령상 근거는 「국민기초생활보장법」제7조 ~ 제10조에 명시되어 있다.

307/124
Question:  근로능력이 있는 수급자에게 생계급여를 실시하는 방법은? | Context Number | 3863
0.4698795180722891 | Answer:  생계급여는 근로능력이 있는 수급자에게 의복, 음식물 및 연료비와 그밖에 일상생활에 기본적으로 필요한 금품을 지급하여 그 생계를 유지하게 하는 것이다.
Before:  None  |  None
REAL Answer:  보장기관은 대통령령으로 정하는 바에 따라 근로능력이 있는 수급자에게 자활에 필요한 사업에 참가할 것을 조건으로 하여 생계급여를 실시할 수 있으며, 이 경우 보장기관은 제28조에 따른 자활지원계획을 고려하여 조건을 제시하여야 합니다.



c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


321/124
Question:  사업근거는 어떤 법령에 근거하고 있나요? | Context Number | 4643
0.8378378378378378 | Answer:  사업근거는 주택도시기금법 제9조제1항제9호, 민간임대주택에 관한 특별법 제4조입니다.
Before:  None  |  None
REAL Answer:  - 주택도시기금법 제9조제1항제9호
- 민간임대주택에 관한 특별법 제4조

322/124
Question:  민간임대(융자) 사업에서 주거복지로드맵을 발표한 시점은? | Context Number | 4643
0.42857142857142855 | Answer:  주거복지로드맵은 '17.11.29'에 발표되었습니다.
Before:  None  |  None
REAL Answer:  2017년 11월 29일입니다.

323/124
Question:  민간임대(융자) 사업에서 기업형 임대사업자 육성방안은 언제 발표되었고, 그 내용은 무엇인가? | Context Number | 4643
0.294478527607362 | Answer:  민간임대(융자) 사업에서 기업형 임대사업자 육성방안은 2015년 1월 13일 발표되었으며, 그 내용은 주거지원 계층 및 무주택 서민 등의 주거 안정을 위해 장기간 거주 가능하고 임대료도 안정적으로 관리되는 전용면적 85㎡ 이하의 공공지원 민간 임대 주택의 건설 및 매입 자금의 일부를 융자 지원하는 것입니다.
Before:  None  |  None
REAL Answer:  2015년 1월 13일에 민간임대리츠 1만호 공급 계획을 발표하였다.



c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


337/124
Question:  2024년 성과지표의 투입지표는 몇 개로 구성되어 있나요? | Context Number | 6828
0.6666666666666666 | Answer:  2024년 성과지표는 26개로 구성되어 있습니다.
Before:  None  |  None
REAL Answer:  성과지표의 성격에 따라 투입지표는 21개로 구성되어 있습니다.

338/124
Question:  2024년 성과지표의 결과지표는 몇 개로 구성되어 있나요? | Context Number | 7587
0 | Answer:  7
Before:  None  |  None
REAL Answer:  성과지표의 성격에 따라 결과지표는 355개로 구성되어 있습니다.

342/124
Question:  노숙인의사상자 지원 프로그램의 성과지표 중 2024년 노숙인시설 종사자 교육 참여율의 목표치는 몇 퍼센트인가? | Context Number | 8872
0.7777777777777777 | Answer:  노숙인시설 종사자 교육 참여율의 목표치는 2024년 80%입니다.
Before:  None  |  None
REAL Answer:  노숙인시설 종사자 교육 참여율의 목표치는 36%이다.

351/124
Question:  식의약 안전 확보를 위해 과학적 심사·시험연구를 강화한다' 전략목표의 2024년 총 예산은 몇 백만원인가요? | Context Number | 4745
0.29850746268656714 | Answer:  전략목표 IV. 식의약 안전 확보를 위해 과학적 심사·시험연구를 강화한다.의 2024년 총 예산은 156,587 백만원입니다.
Before:  None  |  None
REAL Answer:  156,587백만원

352/124
Question:  국가인권위원회의 '24년 예산안은 얼마인가? | Context Number | 7804
0.3783783783783784 | Answer:  국가인권위원회 '24년 예산은 120,738 백만원입니다.
Be